In [15]:
import json
import os
import numpy as np
import regex as re
import pandas as pd

In [16]:
np.random.seed(0)

In [17]:
train_raw = json.load(open('train_codesearchnet_0.json', 'r'))
print(json.dumps(train_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Expects a height mat as input",
    "code": "@ Override public ImageSource apply ( ImageSource input ) { final int [ ] [ ] pixelMatrix = new int [ 3 ] [ 3 ] ; int w = input . getWidth ( ) ; int h = input . getHeight ( ) ; int [ ] [ ] output = new int [ h ] [ w ] ; for ( int j = 1 ; j < h - 1 ; j ++ ) { for ( int i = 1 ; i < w - 1 ; i ++ ) { pixelMatrix [ 0 ] [ 0 ] = input . getR ( i - 1 , j - 1 ) ; pixelMatrix [ 0 ] [ 1 ] = input . getRGB ( i - 1 , j ) ; pixelMatrix [ 0 ] [ 2 ] = input . getRGB ( i - 1 , j + 1 ) ; pixelMatrix [ 1 ] [ 0 ] = input . getRGB ( i , j - 1 ) ; pixelMatrix [ 1 ] [ 2 ] = input . getRGB ( i , j + 1 ) ; pixelMatrix [ 2 ] [ 0 ] = input . getRGB ( i + 1 , j - 1 ) ; pixelMatrix [ 2 ] [ 1 ] = input . getRGB ( i + 1 , j ) ; pixelMatrix [ 2 ] [ 2 ] = input . getRGB ( i + 1 , j + 1 ) ; int edge = ( int ) convolution ( pixelMatrix ) ; int rgb = ( edge << 16 | edge << 8 | edge ) ; output [ j ] [ i ] = rgb ; } } MatrixSource source = new MatrixS

In [18]:
train = []

for row in train_raw:
    train_dict = {}
    train_dict['id'] = 'train-java-{}'.format(row['idx'])
    train_dict['code'] = row['code']
    train_dict['text'] = row['doc']
    train_dict['label'] = row['label']
    train_dict['raw'] = row['raw']
    train_dict['url'] = row['url']
    train.append(train_dict)

In [19]:
test_raw = json.load(open('dev_codesearchnet.json', 'r'))
print(json.dumps(test_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Makes sure the fast - path emits in order .",
    "code": "protected final void fastPathOrderedEmit ( U value , boolean delayError , Disposable disposable ) { final Observer < ? super V > observer = downstream ; final SimplePlainQueue < U > q = queue ; if ( wip . get ( ) == 0 && wip . compareAndSet ( 0 , 1 ) ) { if ( q . isEmpty ( ) ) { accept ( observer , value ) ; if ( leave ( - 1 ) == 0 ) { return ; } } else { q . offer ( value ) ; } } else { q . offer ( value ) ; if ( ! enter ( ) ) { return ; } } QueueDrainHelper . drainLoop ( q , observer , delayError , disposable , this ) ; }",
    "raw": "protected final void fastPathOrderedEmit(U value, boolean delayError, Disposable disposable) {\n        final Observer<? super V> observer = downstream;\n        final SimplePlainQueue<U> q = queue;\n\n        if (wip.get() == 0 && wip.compareAndSet(0, 1)) {\n            if (q.isEmpty()) {\n                accept(observer, value);\n                if (leave(-1) == 0)

In [20]:
test = []

for row in test_raw:
    test_dict = {}
    test_dict['id'] = 'test-java-{}'.format(row['idx'])
    test_dict['code'] = row['code']
    test_dict['text'] = row['doc']
    test_dict['label'] = row['label']
    test_dict['raw'] = row['raw']
    test_dict['url'] = row['url']
    test.append(test_dict)

In [21]:
np.random.shuffle(train)
np.random.shuffle(test)

valid_size = int(len(train) * 0.1)
valid = train[:valid_size]
train = train[valid_size:]

In [22]:
print(len(train))
print(len(valid))
print(len(test))

148431
16492
10955


In [23]:
with open('data_train.json', 'w') as f:
    json.dump(train, f, indent=2)

with open('data_valid.json', 'w') as f:
    json.dump(valid, f, indent=2)

with open('data_test.json', 'w') as f:
    json.dump(test, f, indent=2)


In [24]:
codebase_raw = json.load(open('dev_codebase.json', 'r'))
print(json.dumps(codebase_raw[0], indent=4))

{
    "idx": 1,
    "doc": "Adds an expression to this Select Clause with support for Constructor Expressions .",
    "code": "public < T > QueryBuilder add ( final String lhsStatement , final Class < T > constructorClass , final String expression ) { StringBuilder itemBuilder = new StringBuilder ( ) ; if ( lhsStatement != null ) { itemBuilder . append ( lhsStatement ) ; itemBuilder . append ( \" \" ) ; } itemBuilder . append ( \"NEW \" ) ; itemBuilder . append ( constructorClass . getName ( ) ) ; itemBuilder . append ( \"(\" ) ; itemBuilder . append ( expression ) ; itemBuilder . append ( \")\" ) ; items . add ( itemBuilder . toString ( ) ) ; return builder ; }",
    "raw": "public <T> QueryBuilder add(final String lhsStatement,\n\t\t\tfinal Class<T> constructorClass, final String expression) {\n\n\t\tStringBuilder itemBuilder = new StringBuilder();\n\t\tif (lhsStatement != null) {\n\t\t\titemBuilder.append(lhsStatement);\n\t\t\titemBuilder.append(\" \");\n\t\t}\n\t\titemBuilder.appen

In [25]:

codebase = []

for row in codebase_raw:
    codebase_dict = {}
    codebase_dict['id'] = 'codebase-java-{}'.format(row['idx'])
    codebase_dict['code'] = row['code']
    codebase_dict['text'] = row['doc']
    codebase_dict['label'] = row['label']
    codebase_dict['raw'] = row['raw']
    codebase_dict['url'] = row['url']
    codebase.append(codebase_dict)

In [26]:
np.random.shuffle(codebase)
print(len(codebase))

29392


In [27]:
with open('data_codebase.json', 'w') as f:
    json.dump(codebase, f, indent=2)

In [28]:
print(json.dumps(codebase[:2], indent=4))

[
    {
        "id": "codebase-java-2560",
        "code": "public Decimal max ( Decimal other ) { if ( ( this == NaN ) || ( other == NaN ) ) { return NaN ; } return ( compareTo ( other ) >= 0 ? this : other ) ; }",
        "text": "Returns the maximum of this {",
        "label": 1,
        "raw": "public Decimal max(Decimal other) {\n        if ((this == NaN) || (other == NaN)) {\n            return NaN;\n        }\n        return (compareTo(other) >= 0 ? this : other);\n    }",
        "url": "https://github.com/mdeverdelhan/ta4j-origins/blob/0441468c09cfd98754497cd9c0f1da9cf70fc4b9/ta4j/src/main/java/eu/verdelhan/ta4j/Decimal.java#L371-L376"
    },
    {
        "id": "codebase-java-18844",
        "code": "public static void setParameterValue ( PreparedStatement ps , int paramIndex , int sqlType , Object inValue ) throws SQLException { setParameterValueInternal ( ps , paramIndex , sqlType , null , null , inValue ) ; }",
        "text": "Set the value for a parameter . The method 